In [ ]:
import cv2
import numpy as np
import easyocr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import pickle
import os
from google.colab import drive

In [ ]:
# === Mount Google Drive ===
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# === Load model and label encoder ===
model_path = '/content/drive/My Drive/ML1_Project/MobileNet/Model6/MobileNetBest_Model.h5'
pkl_path = '/content/drive/My Drive/ML1_Project/MobileNet/Model6/MobileNet_Label_Encoder.pkl'

model = load_model(model_path)
print("✅ Model loaded.")

if os.path.exists(pkl_path):
    with open(pkl_path, 'rb') as f:
        label_map = pickle.load(f)
        index_to_label = {v: k for k, v in label_map.items()}
    print("✅ Label encoder loaded.")
else:
    index_to_label = {0: "Handwritten", 1: "Computerized"}
    print("⚠️ Label encoder not found, using default mapping.")

✅ Model loaded.
✅ Label encoder loaded.


In [ ]:
# === Classification function ===
def classify_text_region(region_img):
    try:
        region_img = cv2.resize(region_img, (224, 224))
        region_img = region_img.astype("float32") / 255.0
        region_img = img_to_array(region_img)
        region_img = np.expand_dims(region_img, axis=0)

        preds = model.predict(region_img)

        if preds.shape[-1] == 1:
            return "Computerized" if preds[0][0] > 0.5 else "Handwritten"
        else:
            class_idx = np.argmax(preds[0])
            return index_to_label.get(class_idx, "Unknown")
    except Exception as e:
        print("❌ Classification error:", e)
        return "Unknown"

In [ ]:
# === OCR + Annotation ===
def AnnotatedTextDetection_EasyOCR_from_array(img):
    reader = easyocr.Reader(['en'], gpu=False)
    results = reader.readtext(img)
    annotated_results = []

    for (bbox, text, conf) in results:
        if conf < 0.3 or text.strip() == "":
            continue

        x1, y1 = map(int, bbox[0])
        x2, y2 = map(int, bbox[2])
        w, h = x2 - x1, y2 - y1

        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        label = classify_text_region(crop)
        annotated_results.append(f"{text.strip()} → {label}")

        color = (0, 255, 0) if label == "Computerized" else (255, 0, 0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)

    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB), "\n".join(annotated_results)

In [ ]:
# === Main image processing function ===
def process_image(input_image):
    img = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    result_img, text_result = AnnotatedTextDetection_EasyOCR_from_array(img)
    return result_img, text_result